In [1]:
from Pointcloud.Modules.Object import Pointcloud
from Pointcloud.Modules.Processor import Processor
from Pointcloud.Modules.Utils import TorchUtils

import torch
import meshplot as mp
import matplotlib.pyplot as plt

objectname = "common-3d-test-models-master/fandisk.obj"
# objectname = "cube.obj"

In [2]:
k = 12
noise_level = 0.0
pc_scale = 1

# Sample pointcloud
pc = Pointcloud.sampleObj(objectname, 2 ** 14)
pc.v = pc.v * pc_scale
pp = Processor(pc)

# Add edges
knn_edge_index = pp.graphBuilder.getKNNEdgeIndex(k)
pp.graph.edge_index = knn_edge_index
l = TorchUtils.averageEdgeLength(pp.graph.pos, knn_edge_index)
lk = l / k ** 0.5

# Calculate normals for noise generation
pp.graphBuilder.setAndFlipNormals(flip=False)

# Generate noise
pp.noise.generateNoise(noise_level, l, noise_type=0, noise_direction=1, keepNormals=True)

# Recalculate normals of noisy points
pp.graphBuilder.setAndFlipNormals(flip=True)

Creating MST..: 100%|██████████| 196608/196608 [00:05<00:00, 39296.03it/s]


In [3]:
avg_num_points = 64
r_scale = (2 * avg_num_points / k) ** 0.5
tau = 16 * lk ** 2 

_graph = pp.graph
_pos = _graph.pos
_selector = pp.selector
radius = TorchUtils.pointcloudRadius(_pos)
r = r_scale * l
# cool = pp.graphBuilder.getPVTDecompositionWithKNN(pp.graphBuilder.getKNNEdgeIndex(64))
selection = _selector.getPointsInRangeSelection(r)
cool = pp.graphBuilder.getPVTDecomposition(torch.vstack(selection._batchIndices()))
avg_num = (selection.slices[1:] - selection.slices[:-1]).float().mean()
print(f"Average number of points in neighbourhood: {avg_num}")
filtered_normals = pp.alignor.getVUFilteredNormals(selection)
features, axes = pp.alignor.getVUDecomposition(
    selection,
    filtered_normals
)
values = features
color = (values < tau).sum(dim=1) % 3
print(color.unique(return_counts=True))
print(f"Range: {r}")
print(f"avg_edge_length: {l}")
print(f"Tau: {tau}")
_ = plt.hist(values[:, 1].numpy(), 50)
_ = plt.hist(values[:, 2].numpy(), 50)
_ = plt.hist(values[:, 0].numpy(), 50)
plot = mp.plot(_pos.numpy(), c=color.numpy(), shading={"point_size": lk * 20})
# _ = plot.add_lines(_pos.numpy(), (_pos + _graph.n * l).numpy())
# _ = plot.add_lines(_pos[color == 2].numpy(), (_pos[color == 2] + filtered_normals[color == 2] * l).numpy(), shading={"line_color": "green"})
_ = plot.add_lines(_pos[color == 2].numpy(), (_pos[color == 2] + cool[color == 2, :, 2] * lk * 10).numpy(), shading={"line_color": "blue"})

RuntimeError: linalg.eigh: A must be batches of square matrices, but they are 3 by 1054470 matrices

In [4]:
_graph = pp.graph
_pos = _graph.pos
_graph.edge_index, _graph.mass = pp.graphBuilder.getLaplacianEdgeIndex()
radius = (_pos - _pos.mean(dim=0, keepdim=True)).max()
classes = pp.getMDFeatures()
_ = mp.plot(pp.graph.pos.numpy(), c=classes.numpy(), shading={"point_size": radius / 6})

Process edges for ring 2: 100%|██████████| 3814/3814 [01:08<00:00, 55.61it/s]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.4139499…

In [5]:
# selection = pp.selector.getMDSelection()
_, _, (eigval, _) = pp.alignor.getMDTransformation(selection)
classes = pp.alignor.getMDFeatures(eigval)
_ = mp.plot(pp.graph.pos.numpy(), c=classes.numpy(), shading={"point_size": 0.5})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.4139499…